In [ ]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

In [ ]:
# Reload the web page and execute this cell
import sys
print("User Current Version:-", sys.version)

User Current Version:- 3.8.18 (default, Sep 11 2023, 13:20:55) 
[GCC 11.2.0]


In [ ]:
!python --version

Python 3.8.18


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/automatic-question-generation-master/

[Errno 2] No such file or directory: 'drive/MyDrive/automatic-question-generation-master/'
/content


In [ ]:
%ls

download_squad.py  evaluate.py  models.py  __pycache__/  README.md  runs/  squad_parse.py  train.py


In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
DIR = os.getcwd()

In [ ]:
!python main/download_squad.py --directory dataset

In [ ]:
!python ../main/squad_parse.py --train_filepath ../dataset/train-v2.0.json --dev_filepath ../dataset/dev-v2.0.json

Processing training...

^C


In [ ]:
%cd main

/content/drive/MyDrive/automatic-question-generation-master/main


In [ ]:
!pip install torch==1.5 spacy==2.2.4 torchtext==0.3.1

In [ ]:
!pip list | grep torch

torch                     1.5.0              
torchtext                 0.3.1              


In [ ]:
!pip install -U numpy

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
from torchtext.vocab import Vectors

from tqdm import tqdm
import argparse
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
train_set_path = os.path.join(DIR, 'results/resultssquad_train.csv')
dev_set_path = os.path.join(DIR, 'results/resultssquad_dev.csv')
test_size = 0.7
save = os.path.join(DIR, 'dataset')
train_set = os.path.join(DIR, 'dataset')
word_vector = 'glove'
batch_size = 128
numberbatch_loc = os.path.join(DIR, 'dataset')
resume = ''
epochs = 10

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
# Split dev dataset into test set and validation set
dev_set = pd.read_csv(dev_set_path)
validation_set, test_set = train_test_split(dev_set, test_size = test_size)

In [ ]:
original_train_set = pd.read_csv(train_set_path)
len(original_train_set)

130319

In [ ]:
original_train_set = pd.read_csv(train_set_path)
original_train_set = original_train_set[:100000]
original_train_set.to_csv(save+'/sample_train.csv', index=False)

In [ ]:
# Saving file names to variables
trainloc = train_set_path
#trainloc = os.path.join(DIR, 'dataset/sample_train.csv')
valloc = save+'/validation_set.csv'
testloc = save+'/test_set.csv'

In [ ]:
# Saving validation and test set to csv file
validation_set.to_csv(valloc, index=False)
test_set.to_csv(testloc, index=False)

In [ ]:
!pip list | grep torchtext

torchtext                 0.3.1              


In [ ]:
# Create Field object
tokenize = lambda x: x.split()
TEXT = data.Field(tokenize=tokenize, lower=False, include_lengths = True, init_token = '<SOS>', eos_token = '<EOS>')
LEX = data.Field(tokenize=tokenize, lower=False, init_token = '<SOS>', eos_token = '<SOS>')
BIO = data.Field(tokenize=tokenize, lower=False, init_token = '<SOS>', eos_token = '<SOS>')

In [ ]:
# Specify Fields in the dataset
fields = [('context', TEXT), ('question', TEXT), ('bio', BIO), ('lex', LEX)]

In [ ]:
# Build the dataset
train_data, valid_data, test_data = data.TabularDataset.splits(path = '',train=trainloc, validation=valloc,
															   test=testloc, fields = fields, format='csv', skip_header=True)

In [ ]:
# Build vocabulary
# MAX_VOCAB_SIZE = 50000
MAX_VOCAB_SIZE = 20000
MIN_COUNT = 5
BATCH_SIZE = batch_size

In [ ]:
if word_vector == 'glove':
	TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE,
                 min_freq=MIN_COUNT, vectors='glove.6B.300d',
                 unk_init=torch.Tensor.normal_)
else:
	cache_ = numberbatch_loc
	vectors = Vectors(name='numberbatch-en-19.08.txt', cache=cache_)
	TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE,
                 min_freq=MIN_COUNT, vectors=vectors,
                 unk_init=torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [02:38, 5.42MB/s]                                                
100%|███████████████████████████████████████████████████▉| 399543/400000 [00:55<00:00, 6019.43it/s]

In [ ]:
BIO.build_vocab(train_data)
LEX.build_vocab(train_data)

# Create a set of iterators for each split
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
     batch_size = BATCH_SIZE,
     sort_within_batch = True,
     sort_key = lambda x:len(x.context),
     device = device)

In [ ]:
pad_idx = TEXT.vocab.stoi['<pad>']
eos_idx = TEXT.vocab.stoi['<EOS>']
sos_idx = TEXT.vocab.stoi['<SOS>']

# Size of embedding_dim should match the dim of pre-trained word embeddings
embedding_dim = 300
hidden_dim = 512
vocab_size = len(TEXT.vocab)

In [ ]:
%ls

download_squad.py  evaluate.py  models.py  __pycache__/  README.md  runs/  squad_parse.py  train.py


In [ ]:
# References: https://medium.com/@adam.wearne/seq2seq-with-pytorch-46dc00ff5164

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import random

class Encoder(nn.Module):

    def __init__(self, hidden_size, embedding_size,
                 embedding, answer_embedding, lexical_embedding, n_layers, dropout):

        super(Encoder, self).__init__()

        # Initialize network parameters
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Embedding layer to be shared with Decoder
        self.embedding = embedding
        self.answer_embedding = answer_embedding
        self.lexical_embedding = lexical_embedding

        # Bidirectional GRU
        self.gru = nn.GRU(embedding_size, hidden_size,
                          num_layers=n_layers,
                          dropout=dropout,
                          bidirectional=True)

    def forward(self, input_sequence, input_lengths, answer_sequence, lexical_sequence):

        # Convert input_sequence to word embeddings
        word_embeddings = self.embedding(input_sequence)
        answer_embeddings = self.answer_embedding(answer_sequence)
        lexical_embeddings = self.lexical_embedding(lexical_sequence)

        # Concatenate word embeddings from all features
        final_embeddings = torch.cat((word_embeddings,answer_embeddings,lexical_embeddings), 0)

        # Pack the sequence of embeddings
        packed_embeddings = nn.utils.rnn.pack_padded_sequence(final_embeddings, input_lengths)

        # Run the packed embeddings through the GRU, and then unpack the sequences
        outputs, hidden = self.gru(packed_embeddings)
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)

        # The ouput of a GRU has shape (seq_len, batch, hidden_size * num_directions)
        # Because the Encoder is bidirectional, combine the results from the
        # forward and reversed sequence by simply adding them together.
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]

        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()

        self.hidden_size = hidden_size

    def dot_score(self, hidden_state, encoder_states):
        # Attention model use the dot product formula as global attention
        return torch.sum(hidden_state * encoder_states, dim=2)

    def forward(self, hidden, encoder_outputs, mask):
        attn_scores = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_scores = attn_scores.t()

        # Apply mask so network does not attend <pad> tokens
        attn_scores = attn_scores.masked_fill(mask == 0, -1e10)

        # Return softmax over attention scores
        return F.softmax(attn_scores, dim=1).unsqueeze(1)

class Decoder(nn.Module):
    def __init__(self, embedding, embedding_size,
                 hidden_size, output_size, n_layers, dropout):

        super(Decoder, self).__init__()

        # Initialize network params
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.embedding = embedding

        self.gru = nn.GRU(embedding_size, hidden_size, n_layers,
                          dropout=dropout)

        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.attn = Attention(hidden_size)

    def forward(self, current_token, hidden_state, encoder_outputs, mask):

        # convert current_token to word_embedding
        embedded = self.embedding(current_token)

        # Pass through GRU
        rnn_output, hidden_state = self.gru(embedded, hidden_state)

        # Calculate attention weights
        attention_weights = self.attn(rnn_output, encoder_outputs, mask)

        # Calculate context vector
        context = attention_weights.bmm(encoder_outputs.transpose(0, 1))

        # Concatenate  context vector and GRU output
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))

        # Pass concat_output to final output layer
        output = self.out(concat_output)

        # Return output and final hidden state
        return output, hidden_state

class Seq2seq(nn.Module):
    def __init__(self, embedding_size, hidden_size, vocab_size,
                 device, pad_idx, eos_idx, sos_idx, teacher_forcing_ratio=0.5):
        super(Seq2seq, self).__init__()

        # Initialize embedding layer shared by encoder and decoder
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.answer_embedding = nn.Embedding(6, embedding_size, padding_idx=1)
        # Size could sometime change, depend on the device that the model is trained on
        self.lexical_embedding = nn.Embedding(452, embedding_size, padding_idx=1)

        # Encoder network
        self.encoder = Encoder(hidden_size,
                               embedding_size,
                               self.embedding,
                               self.answer_embedding,
                               self.lexical_embedding,
                               n_layers=2,
                               dropout=0.5)

        # Decoder network
        self.decoder = Decoder(self.embedding,
                               embedding_size,
                               hidden_size,
                               vocab_size,
                               n_layers=2,
                               dropout=0.5)


        # Indices of special tokens and hardware device
        self.pad_idx = pad_idx
        self.eos_idx = eos_idx
        self.sos_idx = sos_idx
        self.device = device

    def create_mask(self, input_sequence):

        return (input_sequence != self.pad_idx).permute(1, 0)

    def forward(self, input_sequence, answer_sequence, lexical_sequence, output_sequence, teacher_forcing_ratio):

        # Unpack input_sequence tuple
        input_tokens = input_sequence[0]
        input_lengths = input_sequence[1]

        # Unpack output_tokens, or create an empty tensor for text generation
        if output_sequence is None:
            inference = True
            output_tokens = torch.zeros((100, input_tokens.shape[1])).long().fill_(self.sos_idx).to(self.device)
        else:
            inference = False
            output_tokens = output_sequence[0]

        vocab_size = self.decoder.output_size

        batch_size = len(input_lengths)
        max_seq_len = len(output_tokens)

        # Tensor initialization to store Decoder output
        outputs = torch.zeros(max_seq_len, batch_size, vocab_size).to(self.device)

        # Pass through the first half of the network
        encoder_outputs, hidden = self.encoder(input_tokens, input_lengths, answer_sequence, lexical_sequence)

        # Ensure dim of hidden_state can be fed into Decoder
        hidden =  hidden[:self.decoder.n_layers]

        # First input to the decoder is the <sos> tokens
        output = output_tokens[0,:]

        # Create mask
        mask = self.create_mask(input_tokens)

        # Step through the length of the output sequence one token at a time
        # Teacher forcing is used to assist training
        for t in range(1, max_seq_len):
            output = output.unsqueeze(0)

            output, hidden = self.decoder(output, hidden, encoder_outputs, mask)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (output_tokens[t] if teacher_force else top1)

            # If we're in inference mode, keep generating until we produce an
            # <eos> token
            if inference and output.item() == self.eos_idx:
                return outputs[:t]

        return outputs

In [ ]:
# Initializing weights
# from models import Seq2seq
model = Seq2seq(embedding_dim, hidden_dim, vocab_size, device, pad_idx, eos_idx, sos_idx).to(device)

100%|███████████████████████████████████████████████████▉| 399543/400000 [01:10<00:00, 6019.43it/s]

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-4.5283e-01,  6.7827e-01, -6.2774e-01,  ...,  2.0826e+00,
          8.9462e-01,  1.0935e+00],
        [-1.4745e+00,  6.2370e-01, -1.8066e+00,  ...,  4.8404e-02,
          1.0767e+00, -1.6317e+00],
        [ 6.6187e-01,  4.3202e-01, -1.6619e+00,  ...,  4.9365e-01,
          3.8861e-01, -5.3126e-01],
        ...,
        [ 5.3634e-01,  1.9325e-03,  2.5228e-02,  ..., -6.6040e-02,
         -2.3108e-01,  3.4183e-01],
        [-1.7536e-01,  6.6148e-01, -6.2990e-01,  ..., -1.5426e+00,
          6.9536e-01, -7.2572e-01],
        [-1.4161e+00,  1.2042e-01,  6.2281e-01,  ..., -7.3627e-01,
          1.0697e-01, -4.3506e-01]], device='cuda:0')

In [ ]:
# Initializing weights for special tokens
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

In [ ]:
model.embedding.weight.requires_grad = False

optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True],
                       lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

In [ ]:
save

'/content/drive/MyDrive/automatic-question-generation-master/dataset'

In [ ]:
# If continuing training
# resume = os.path.join(save, 'model.pth')
if (resume):
	model.load_state_dict(torch.load(resume))

In [ ]:
!pip install tensorboard==1.14

In [ ]:
!pip install --upgrade --force-reinstall protobuf==3.7.0rc2

In [ ]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
def train(model, iterator, criterion, optimizer, clip):
    # Put the model in training mode
    model.train()

    epoch_loss = 0

    for idx, batch in tqdm(enumerate(iterator), total=len(iterator)):

        input_sequence = batch.context
        answer_sequence = batch.bio
        output_sequence = batch.question
        lexical_sequence = batch.lex

        target_tokens = output_sequence[0]


        # zero out the gradient for the current batch
        optimizer.zero_grad()

        # Run the batch through the model
        output = model(input_sequence, answer_sequence, lexical_sequence, output_sequence, 0.5)

        # Throw it through the loss function
        output = output[1:].view(-1, output.shape[-1])
        target_tokens = target_tokens[1:].view(-1)

        loss = criterion(output, target_tokens)
        #writer.add_scalar("Loss/train", loss, epoch)

        # Perform back-prop and calculate the gradient of the loss function
        loss.backward()

        # Clip the gradient if necessary.
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Update model parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    # Put model in evaluation mode
    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for idx, batch in tqdm(enumerate(iterator), total=len(iterator)):

            input_sequence = batch.context
            answer_sequence = batch.bio
            output_sequence = batch.question
            lexical_sequence = batch.lex

            target_tokens = output_sequence[0]

            # Run the batch through the model
            output = model(input_sequence, answer_sequence, lexical_sequence, output_sequence, 0)

            # Throw it through the loss function
            output = output[1:].view(-1, output.shape[-1])
            target_tokens = target_tokens[1:].view(-1)

            loss = criterion(output, target_tokens)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
N_EPOCHS = 50
CLIP = 1


In [ ]:
epochs

10

In [ ]:
best_valid_loss = float('inf')

In [ ]:
i=0
for epoch in range(N_EPOCHS):

    train_loss = train(model, train_iterator, criterion, optimizer, CLIP)
    #writer.flush()
    valid_loss = evaluate(model, valid_iterator, criterion)

    if i%10 ==0:
      torch.save(model.state_dict(), save+f'/model_{i}.pth')
    i+=1

    # if valid_loss < best_valid_loss:
        # best_valid_loss = valid_loss
        # torch.save(model.state_dict(), save+'/model.pth')

    print('Epoch: ', epoch)
    print('Train loss: ', train_loss)
    print('Valid loss: ', valid_loss)

NameError: ignored

In [ ]:
test_loss = evaluate(model, test_iterator, criterion)

print('Test Loss: {:.2f}'.format(test_loss))

for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

In [ ]:
resume

'/content/drive/MyDrive/automatic-question-generation-master/dataset/model.pth'

In [ ]:
!pip install -U spacy

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
%ls

download_squad.py  evaluate.py  models.py  __pycache__/  README.md  runs/  squad_parse.py  train.py


In [ ]:
model.load_state_dict(torch.load(resume))

<All keys matched successfully>

In [ ]:
import spacy

In [ ]:
spacy.load('en_core_web_sm')

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
def translate_sentence(model, sentence, nlp):
    model.eval()

    tokenized = nlp(sentence)

    tokenized = ['<sos>'] + [t.lower_ for t in tokenized] + ['<eos>']
    numericalized = [TEXT.vocab.stoi[t] for t in tokenized]

    sentence_length = torch.LongTensor([len(numericalized)]).to(model.device)
    tensor = torch.LongTensor(numericalized).unsqueeze(1).to(model.device)

    translation_tensor_logits = model((tensor, sentence_length), None, 0)

    translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1)
    translation = [TEXT.vocab.itos[t] for t in translation_tensor]

    # Start at the first index.  We don't need to return the <sos> token...
    translation = translation[1:]
    return translation, translation_tensor_logits
sentence = "tell me a fun fact"
response, logits = translate_sentence(model, sentence, nlp)
print(" ".join(response))


In [ ]:
import spacy
import torch

nlp = spacy.load('en_core_web_sm')

# Assuming you have a translation model defined and initialized as 'model'

def translate_sentence(model, sentence, nlp):
    model.eval()

    tokenized = nlp(sentence)

    tokenized = ['<sos>'] + [t.lower_ for t in tokenized] + ['<eos>']
    numericalized = [TEXT.vocab.stoi[t] for t in tokenized]

    sentence_length = torch.LongTensor([len(numericalized)]).to(model.device)
    tensor = torch.LongTensor(numericalized).unsqueeze(1).to(model.device)

    # You need to provide placeholders for 'output_sequence' and 'teacher_forcing_ratio'
    output_sequence = torch.zeros_like(tensor)  # Use a tensor of zeros as a placeholder
    teacher_forcing_ratio = 0.5  # Replace with your desired teacher forcing ratio

    translation_tensor_logits = model((tensor, sentence_length), output_sequence, teacher_forcing_ratio)

    translation_tensor = torch.argmax(translation_tensor_logits.squeeze(1), 1)
    translation = [TEXT.vocab.itos[t] for t in translation_tensor]

    # Start at the first index. We don't need to return the <sos> token...
    translation = translation[1:]
    return translation, translation_tensor_logits

sentence = "tell me a fun fact"
response, logits = translate_sentence(model, sentence, nlp)
print(" ".join(response))


TypeError: ignored

In [ ]:
# Code adapted from : https://github.com/bentrevett/pytorch-seq2seq

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
from torchtext.vocab import Vectors

from tqdm import tqdm
from models import Seq2seq
import random
import pandas as pd
import numpy as np

import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import single_meteor_score
## Wordnet dependencies from meteor score
#nltk.download('wordnet')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load data
trainloc = os.path.join(DIR, 'results/resultssquad_train.csv')
valloc = os.path.join(DIR, 'dataset/validation_set.csv')
testloc = os.path.join(DIR, 'dataset/test_set.csv')

# Create Field object
tokenize = lambda x: x.split()
TEXT = data.Field(tokenize=tokenize, lower=False, include_lengths = True, init_token = '<SOS>', eos_token = '<EOS>')
LEX = data.Field(tokenize=tokenize, lower=False, init_token = '<SOS>', eos_token = '<SOS>')
BIO = data.Field(tokenize=tokenize, lower=False, init_token = '<SOS>', eos_token = '<SOS>')

# Specify Fields in the dataset
fields = [('context', TEXT), ('question', TEXT), ('bio', BIO), ('lex', LEX)]

# Build the dataset
train_data, valid_data, test_data = data.TabularDataset.splits(path = '',train=trainloc, validation=valloc,
                                 test=testloc, fields = fields, format='csv', skip_header=True)

# Build vocabulary
MAX_VOCAB_SIZE = 50000
MIN_COUNT = 5
BATCH_SIZE = 128

if 'glove' == 'glove':
  TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE,
                 min_freq=MIN_COUNT, vectors='glove.6B.300d',
                 unk_init=torch.Tensor.normal_)
else:
  cache_ = os.path.join(DIR, 'dataset')
  vectors = Vectors(name='numberbatch-en-19.08.txt', cache=cache_)
  TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE,
                 min_freq=MIN_COUNT, vectors=vectors,
                 unk_init=torch.Tensor.normal_)

BIO.build_vocab(train_data)
LEX.build_vocab(train_data)

# Building model
pad_idx = TEXT.vocab.stoi['<pad>']
eos_idx = TEXT.vocab.stoi['<EOS>']
sos_idx = TEXT.vocab.stoi['<SOS>']

# Size of embedding_dim should match the dim of pre-trained word embeddings
embedding_dim = 300
hidden_dim = 512
vocab_size = len(TEXT.vocab)

# Initializing weights
model = Seq2seq(embedding_dim, hidden_dim, vocab_size, device, pad_idx, eos_idx, sos_idx).to(device)
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

# Initializing weights for special tokens
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[pad_idx] = torch.zeros(embedding_dim)

model.embedding.weight.requires_grad = False

optimizer = optim.Adam([param for param in model.parameters() if param.requires_grad == True],
                       lr=1.0e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

# Load model
model.load_state_dict(torch.load(resume))

def predict_question(model, paragraph, answer_pos, lex_features):
    model.eval()

    tokenized = ['<SOS>'] + paragraph + ['<EOS>']
    numericalized = [TEXT.vocab.stoi[t] for t in tokenized]

    tokenized_answer = ['<SOS>'] + answer_pos + ['<EOS>']
    numericalized_answer = [BIO.vocab.stoi[t] for t in tokenized_answer]

    tokenized_lex = ['<SOS>'] + lex_features + ['<EOS>']
    numericalized_lex = [LEX.vocab.stoi[t] for t in tokenized_lex]

    paragraph_length = torch.LongTensor([len(numericalized)]).to(model.device)
    tensor = torch.LongTensor(numericalized).unsqueeze(1).to(model.device)

    answer_tensor = torch.LongTensor(numericalized_answer).unsqueeze(1).to(model.device)
    lex_tensor = torch.LongTensor(numericalized_lex).unsqueeze(1).to(model.device)

    question_tensor_logits = model((tensor, paragraph_length), answer_tensor, lex_tensor, None, 0)

    question_tensor = torch.argmax(question_tensor_logits.squeeze(1), 1)
    question = [TEXT.vocab.itos[t] for t in question_tensor]

    # Start at the first index.  We don't need to return the <SOS> token
    question = question[1:]

    return question, question_tensor_logits

# Display prediction
num = 100
example_idx = random.sample(range(1,300),num)

for i in example_idx:
  src = vars(train_data.examples[i])['context']
  trg = vars(train_data.examples[i])['question']
  ans = vars(train_data.examples[i])['bio']
  lex = vars(train_data.examples[i])['lex']

  print('context: ', ' '.join(src))
  print('question: ', ' '.join(trg))

  question, logits = predict_question(model, src, ans, lex)
  print('predicted: ', " ".join(question))
  print()

for j in example_idx:
  src = vars(test_data.examples[j])['context']
  trg = vars(test_data.examples[j])['question']
  ans = vars(test_data.examples[j])['bio']
  lex = vars(test_data.examples[j])['lex']

  print('context: ', ' '.join(src))
  print('question: ', ' '.join(trg))

  question, logits = predict_question(model, src, ans, lex)
  print('predicted: ', " ".join(question))
  print()



In [ ]:
def calculate_bleu_and_meteor(data, model):

    trgs = []
    pred_trgs = []
    meteor_score_ = []

    for datum in data:

        src = vars(datum)['context']
        trg = vars(datum)['question']
        ans = vars(datum)['bio']
        lex = vars(datum)['lex']

        pred_trg, _ = predict_question(model, src, ans, lex)

        #cut off <EOS> token
        pred_trg = pred_trg[:-1]

        pred_trgs.append(pred_trg)
        # print(pred_trg)
        trgs.append(trg)
        # print(trg)
        meteor_score_.append(single_meteor_score(' '.join(pred_trg),' '.join(trg)))

    bleu_score = corpus_bleu(pred_trgs, trgs)
    meteor_score_ = np.mean(meteor_score_)

    return bleu_score,meteor_score_

bleu_score, meteor_score_ = calculate_bleu_and_meteor(test_data, model)

print('BLEU score = {:.2f}'.format(bleu_score*100))
print('METEOR score = {:.2f}'.format(meteor_score_*100))

TypeError: ignored